In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
import json
from copy import deepcopy

from torch import optim
from torch.utils.data import DataLoader

from data.datasets.random_dataset import RandomDataset
from data.datasets.golden_panels import GoldenPanelsDataset

from networks.plain_ssupervae import PlainSSuperVAE
from networks.ssupervae_contextual_attentional import SSuperVAEContextualAttentional
from training.ssupervae_contextual_attn_trainer import SSuperVAEContextualAttentionalTrainer
from training.vae_trainer import VAETrainer
from utils.config_utils import read_config, Config
from utils.plot_utils import *
from utils.logging_utils import *
from utils import pytorch_util as ptu
from utils.image_utils import *
from configs.base_config import *
from functional.losses.elbo import elbo

In [5]:
initiate_logger()

In [6]:
def save_best_loss_model(model_name, model, best_loss):
    print('[INFO] Current best loss: ' + str(best_loss))
    torch.save(model, base_dir + 'playground/ssupervae/weights/' + model_name + ".pth")


def train(data_loader,
          config,
          panel_dim,
          model_name='plain_ssupervae',
          cont_epoch=-1,
          cont_model=None):
    # loading config
    print("[INFO] Initiate training...")

    # creating model and training details
    net = SSuperVAEContextualAttentional(config.backbone,
                                         panel_img_size=panel_dim,
                                         latent_dim=config.latent_dim,
                                         embed_dim=config.embed_dim,
                                         seq_size=config.seq_size,
                                         decoder_channels=config.decoder_channels,
                                         gen_img_size=config.image_dim).to(ptu.device)

    criterion = elbo

    optimizer = optim.Adam(net.parameters(),
                           lr=config.lr,
                           betas=(config.beta_1, config.beta_2),
                           weight_decay=config.weight_decay)

    d_params = list(net.local_disc.parameters()) + list(net.global_disc.parameters())
    optimizer_disc = optim.Adam(d_params,
                                lr=config.lr,
                                betas=(config.beta_1, config.beta_2),
                                weight_decay=config.weight_decay)

    scheduler = optim.lr_scheduler.LambdaLR(optimizer,
                                            lambda epoch: (config.train_epochs - epoch) / config.train_epochs,
                                            last_epoch=-1)

    scheduler_disc = optim.lr_scheduler.LambdaLR(optimizer_disc,
                                            lambda epoch: (config.train_epochs - epoch) / config.train_epochs,
                                            last_epoch=-1)
    # init trainer
    trainer = SSuperVAEContextualAttentionalTrainer(model=net,
                                                    config_disc=config,
                                                    model_name=model_name,
                                                    criterion=criterion,
                                                    train_loader=data_loader,
                                                    test_loader=None,
                                                    epochs=config.train_epochs,
                                                    optimizer=optimizer,
                                                    optimizer_disc=optimizer_disc,
                                                    scheduler=scheduler,
                                                    scheduler_disc=scheduler_disc,
                                                    grad_clip=config.g_clip,
                                                    best_loss_action=lambda m, l: save_best_loss_model(model_name, m,
                                                                                                       l),
                                                    save_dir=base_dir + 'playground/ssupervae/',
                                                    checkpoint_every_epoch=True
                                                    )

    if cont_epoch > -1:
        epoch, losses = trainer.load_checkpoint(epoch=cont_epoch)
    elif cont_model is not None:
        epoch, losses = trainer.load_checkpoint(alternative_chkpt_path=cont_model)
        print("[INFO] Continues from loaded model in epoch:", epoch)
        scheduler.step()
    else:
        epoch, losses = None, {}

    train_losses, test_losses = trainer.train_epochs(starting_epoch=epoch, losses=losses)

    print("[INFO] Completed training!")

    save_training_plot(train_losses['loss'],
                       test_losses['loss'],
                       "Plain_SSuperVAE Losses",
                       base_dir + 'playground/supervae/' + f'results/ssupervae_plot.png'
                       )
    return net

In [7]:
ptu.set_gpu_mode(True)
config = read_config(Config.VAE_CONTEXT_ATTN)
golden_age_config = read_config(Config.GOLDEN_AGE)

panel_dim = golden_age_config.panel_dim[0]

cont_epoch = -1
cont_model = None  # "playground/ssupervae/weights/model-18.pth"
# TODO: move this to config
limit_size = -1

# data = RandomDataset((3, 3, 360, 360), (3, config.image_dim, config.image_dim))
data = GoldenPanelsDataset(golden_age_config.panel_path,
                           golden_age_config.sequence_path,
                           golden_age_config.panel_dim,
                           config.image_dim,
                           augment=False,
                           mask_val=golden_age_config.mask_val,
                           mask_all=golden_age_config.mask_all,
                           return_mask=golden_age_config.return_mask,
                           train_test_ratio=golden_age_config.train_test_ratio,
                           train_mode=True,
                           limit_size=limit_size)
data_loader = DataLoader(data, batch_size=config.batch_size, shuffle=True, num_workers=4)

In [ ]:
model = train(data_loader,
              config,
              model_name=get_dt_string() + "_model",
              cont_epoch=cont_epoch,
              cont_model=cont_model,
              panel_dim=panel_dim)
torch.save(model, base_dir + 'playground/ssupervae/results/' + "ssuper_vae_context_model.pth")

[INFO] Initiate training...
Loaded pretrained weights for efficientnet-b5


  0%|          | 0/18016 [00:00<?, ?it/s]/kuacc/users/gsoykan20/.conda/envs/ulad/lib/python3.6/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
Epoch 0, loss 17703.8775, reconstruction_loss 18153.0316, kl_loss 19.0924, l1_fine 1.0409, wgan_g -469.2875, wgan_d -509.7567, wgan_gp 19.8764, d -310.9925: 100%|██████████| 18016/18016 [1:57:59<00:00,  2.54it/s]


[INFO] Current best loss: 0


/kuacc/users/gsoykan20/.conda/envs/ulad/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
Epoch 1, loss 17606.2888, reconstruction_loss 18006.1413, kl_loss 30.2759, l1_fine 1.0372, wgan_g -431.1657, wgan_d -531.1582, wgan_gp 20.4321, d -326.8370: 100%|██████████| 18016/18016 [1:57:14<00:00,  2.56it/s]
Epoch 2, loss 17079.8024, reconstruction_loss 17378.4287, kl_loss 38.3255, l1_fine 0.9827, wgan_g -337.9343, wgan_d -493.6751, wgan_gp 18.7416, d -306.2594:  41%|████      | 7428/18016 [48:45<1:09:11,  2.55it/s]

In [ ]:
# experiment start time: may 14 - 18.04